# NLP前的数据预处理
之前已经分享了基础的数据预处理方法，更准确地说是数据的读写。但真正做NLP的时候并没有那么简单，读取完数据之后，还需要很重要的一步预处理工作，才能放到模型里去跑，例如分词、编码，这里可以理解为是真正的NLP操作！之前能跑BERT是因为BERT已经囊括了预处理工作，可以发现源码里的`tokenization.py`/`optimization.py`/`modeling.py`都是主函数的依赖库，其中`tokenization.py`里的方法就负责NLP数据预处理。

以下你将会看到：
* 标准化
* 分词
* 词汇表
* torchtext

## 标准化
如果你拿到的数据比较混乱。。。比如大小写、标点、罗马数字，所以你最开始得先对语料做个normalization，可能会涉及到，转义、半角/全角化/去除标点、添加特殊符号等。其实都比较简单，我随便举个例子，比如半/全角化。

In [11]:
def full2half(text):
    output_text = ''
    for t in text:
        code = ord(t)
        if code == 0x300:
            code = 32
        elif 0xff01 <= code <= 0xff5e:
            code -= 0xfee0
        output_text += chr(code)
    return output_text

def half2full(text):
    output_text = ''
    for t in text:
        code = ord(t)
        if code == 32:
            code = 0x300
        elif 33 <= code <= 126:
            code += 0xfee0
        output_text += chr(code)
    return output_text

## 分词
然后就是分词，一般NLP都得分词，尤其是中文。分词是切分语料的一种方法，其它还有分字和分句。

**分字**

分字属于最细粒度的操作，虽然英文里还有更细的分法，因为有词根啥的，但中文里应该没有，比如：
```txt
我是一个机器人。
我|是|一|个|机|器|人|。
```
是的，等于没分。

**分词**

上述操作可以用split来实现，然而没什么卵用，比如`爸`和`爸爸`两个词基本上相等，`爸`并不等于`爸`x2。所以分词很重要，能帮助模型更好地理解语义，很大程度上会左右模型的性能。

**分句**

这个粒度就比较粗了，在某些句子级的任务里可能会用到。

所以一般来说中等粒度的分词是最合适的，尤其是对中文，如下：
```txt
我|是|一|个|机器|人|。
我|是|一|个|机器人|。
我|是|一个|机器人|。
```
这三种分词都是对的，最后一个应该相对好一些，具体怎么分取决于你，和你用的工具。。。

↓↓↓

进入码农时刻，以下分享一些比较好用的分词工具：
* jieba
* LTP
* thulac
* pkuseg

其实很多人应该都听过spaCy，是一款功能很强大且齐全的NLP处理包，里面就包括了分词，用法可以参考[这里](https://spacy.io/models/)。但它不支持中文，这里就不多讲了，刚也说了中文分词和英文是不一样滴。

### jieba

jieba可能是中文分词里最有名的，所有工具都会拿它来做比较，因为它永远是最后一名。。。呸，因为jieba人家很好用啊，性能嘛，还可以。。。

In [2]:
import jieba

text1 = '啊，我是一个机器人。'

seg_default = jieba.cut(text1)
seg_all = jieba.cut(text1, cut_all=True)
seg_search = jieba.cut_for_search(text1)

print('|'.join(list(seg_default)))
print('|'.join(list(seg_all)))
print('|'.join(list(seg_search)))

啊|，|我|是|一个|机器人|。
啊|，|我|是|一个|机器|机器人|。
啊|，|我|是|一个|机器|机器人|。


jieba分词内置有三种模式：
* 默认模式：分词
* 全模式：列出所有成词的可能
* 搜索引擎模式：粒度更加细，适合搜索引擎召回（？？？）

注意：jieba返回的分词结果是一个迭代器，需要转换成列表才能打印，也可以直接调用lcut和lcut_for_search，直接返回数组。

一般来说分词用默认模式就够，你看jieba分得很好嘛不是，那么看下面一段话。

In [3]:
text2 = '塞雷娅，前莱茵生命防卫科主任。于生命科学、微生物学、源石技艺等领域皆有建树，\
同时于歼灭战、要员保全、异常事态处理等任务中表现出强大实力。\
目前与罗德岛在科研等多个领域皆有合作，同时，正与罗德岛针对深入合作计划进行磋商。'

seg = jieba.lcut(text2)

print('|'.join(seg))

塞雷娅|，|前|莱茵|生命|防卫|科主任|。|于|生命科学|、|微生物学|、|源石|技艺|等|领域|皆|有|建树|，|同时|于|歼灭战|、|要员|保全|、|异常|事态|处理|等|任务|中|表现|出|强大|实力|。|目前|与|罗德岛|在|科研|等|多个|领域|皆|有|合作|，|同时|，|正|与|罗德岛|针对|深入|合作|计划|进行|磋商|。


这次其实分得就有些问题了，这段话摘自明日方舟游戏里的档案，因为是游戏，里面的特殊词jieba根本识别不出来，比如这些其实是一个词：
```txt
莱茵生命
防卫科
源石技艺
要员保全
异常事态处理
```
这问题其实可以解决，为了适应用户特定语料的需求，比如金融、体育、游戏，jieba允许用户针对自己的语料添加词典，词典为txt格式，每行一个词，就像上面那样。

接下来我们添加一下词典再看看效果如何。

In [74]:
dict_dir = 'data/dict.txt'
jieba.load_userdict(dict_dir)

seg = jieba.lcut(text2)

print('|'.join(seg))

塞雷娅|，|前|莱茵生命|防卫科|主任|。|于|生命科学|、|微生物学|、|源石技艺|等|领域|皆|有|建树|，|同时|于|歼灭战|、|要员保全|、|异常事态处理|等|任务|中|表现|出|强大|实力|。|目前|与|罗德岛|在|科研|等|多个|领域|皆|有|合作|，|同时|，|正|与|罗德岛|针对|深入|合作|计划|进行|磋商|。


没错，这就对了！jieba是不是用起来很方便，当然它除了分词还支持些别的功能，比如统计词频、特征抽取、词性标注，以下简单举个例子，更多方法可以看下[源码](https://github.com/fxsjy/jieba)。

**特征抽取**

In [4]:
import jieba.analyse

features = jieba.analyse.extract_tags(text2, topK=10, withWeight=True)

for word, score in features:
    print(word, score)

这里jieba用的是TF-IDF，它基于统计逆频率，简单来说就是一个词在本段里出现得越多，而在其它地方出现得越少，那么它对段落就越重要，是一种比较经典的特征抽取算法。参数topK表示要抽的特征数量，withWeight=True将会输出关键词对应的TF-IDF得分。

**词性标注**

In [29]:
import jieba.posseg

tags = jieba.posseg.lcut(text1)

for word, tag in tags:
    print(word, tag)

以上jieba就分享到这里，可以感受到，jieba设计得还是比较友好，效果也挺好，觉得效果不理想就手动添加词典。

### LTP
这是哈工大的一款语言技术平台，性能会比jieba好不少，但用起来麻烦点。
```bash
pip install pyltp
```
pyltp是Python中的接口，因为它源码是用C++写的，因此Python上安装的时候会涉及到依赖问题，如果你没报错那最好。。。

其次是用pyltp需要下载预训练模型，就跟spaCy一样，[最新版模型](http://ltp.ai/download.html)。

接下来就可以分词了，我这里应该没有放模型，你可以脑补一下有这样一个路径。

In [32]:
from pyltp import Segmentor

model_dir = 'ltp_data_v3.4.0/cws.model'

model = Segmentor()
model.load(model_dir)

seg = model.segment(text2)

print('|'.join(list(seg)))

问题还是一样，我们同样可以通过添加词典解决。

In [41]:
model.load_with_lexicon(model_dir, 'data/dict.txt')

seg = model.segment(text2)

print('|'.join(list(seg)))
model.release()

塞雷娅|，|前|莱茵生命|防卫科|主任|。|于|生命|科学|、|微生物学|、|源石技艺|等|领域|皆|有|建树|，|同时|于|歼灭战|、|要员保全|、|异常|事态|处理|等|任务|中|表现|出|强大|实力|。|目前|与|罗德岛|在|科研|等|多|个|领域|皆|有|合作|，|同时|，|正|与|罗德岛|针对|深入|合作|计划|进行|磋商|。


LTP虽然很强大，但Python上用起来不太友好，其次用它得下模型，我家网最近不是很好。。。

所以更加推荐接下来两个包，thulac和pkuseg，分别来自清华和北大，用法很接近，而且不用额外下载模型。（×）

就来看看你们谁更nb吧~
### thulac
奇怪的是用清华源下不动。。。亲测豆瓣源比较稳。
```bash
pip install -i https://pypi.douban.com/simple/ thulac
```

In [54]:
import thulac

model = thulac.thulac(user_dict='data/dict.txt', model_path=None, T2S=False, seg_only=True, filt=False, deli='_')

seg = model.cut(text2)

print('|'.join(s[0] for s in seg))

参数里同样可以使用自己的user_dict，用户可以通过model_path指定预训练模型，默认为None，T2S=True将自动把繁体转成简体，seg_only=False时模型会自动标注词性，deli设定打印的时候词和词性之间的分隔符，filt=True时模型会自动过滤掉一些无意义的词。

### pkuseg
```bash
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pkuseg
```

In [4]:
import pkuseg

model = pkuseg.pkuseg(user_dict='data/dict.txt')

seg = model.cut(text2)

print('|'.join(seg))

塞雷娅|，|前|莱茵生命|防卫科|主任|。|于|生命|科学|、|微生物学|、|源石技艺|等|领域|皆|有|建树|，|同时|于|歼灭战|、|要员保全|、|异常事态处理|等|任务|中|表现|出|强大|实力|。|目前|与|罗德岛|在|科研|等|多|个|领域|皆|有|合作|，|同时|，|正|与|罗德岛|针对|深入|合作|计划|进行|磋商|。


目前其实感觉两个包差不多，但pkuseg的特色是可以细分领域，目前已有的领域词模型有news、web、medicine、tourism，都是已经训练好的模型。

注意：pip会自动下载默认的混合领域模型mixed。

In [ ]:
model = pkuseg.pkuseg(model_name='medicine')

seg = model.cut(text2)

print('|'.join(seg))

另外pkuseg还支持用自己的语料训练一个分词模型，最终输出训练好的features和weights，输入格式好像txt和csv都可以，这地方没太研究过。

In [18]:
trainfile_dir = 'data/trainfile.txt'
testfile_dir = 'data/testfile.txt'
output_dir = 'data'

pkuseg.train(trainfile_dir, testfile_dir, output_dir, train_iter=10, init_model=None)

两个包的性能都很好，连操作都一样。。。缺点就是写得有点简陋。。。（偷懒的下场）不过既然是开源的，自己做点修改也无妨。

你可能还听到过一个HanLP，也是很牛逼的一个包，但人家是用java写的，Python上想用一样很烦，因此同样不太推荐用pyltp。以及如果你分词只是为了预处理，其实不必太纠结于性能。根据你的语料库不同，分词工具未必会识别得出来你想要得词，但这个问题可以通过添加词典来弥补，所以并不非得用一个普适意义上性能很好的分词工具。

↓↓↓

推荐使用pkuseg或者jieba，真香。
## 词汇表
这里就是构建所谓的vocabulary，里面包含了所有语料里有用的词，包括语料本身的词，和一些我们后来附加的标记。可以参考BERT模型里的vocab.txt文件，里面标记比较复杂，一般来说我们会给语料附加上这几种：
* &lt;sos&gt;：句首
* &lt;eos&gt;：句尾
* &lt;pad&gt;：填补位
* &lt;unk&gt;：未知词

随后我们需要给vocab里的每个词赋一个唯一的id，通常用one-hot或者自然数。除此之外还需要给每个词赋予一个词向量，因为我们不可能真正把一个字送到模型里去，而是告诉它id，模型通过这些id到查找表里找出对应的词向量。这里的词向量可以用预训练好的，比如GolVe、word2vec、BERT，也可以随机初始化。

构建词表这一步可以用字典操作来实现，这里就先不展示了，我们直接进入torchtext。
## torchtext
torchtext跟torchvision一样，是专门用来做NLP预处理的，目前好像用的人不太多，我同事表示这是啥？？没听过。。新手刚开始用会有点不太习惯，但用了几次你就会发现是个神器。torchtext里面自带有分词、词频、数据集切分等步骤，甚至还有现成的数据集给你来玩玩。。。

**Field**

Field对象用来定义如何处理你的数据，大概有这么几个参数：
* sequential：是否需要分词，默认True
* use_vocab：是需要生成id，默认True
* tokenize：可以指定用作分词的方法，默认是split
* init_token：句首的占位符
* eos_token：句尾的占位符
* pad_token：padding的占位符，默认'&lt;pad&gt;'
* unk_token：默认'&lt;unk&gt;'
* fix_length：是否定长，会自动帮你做padding，默认None，也就是变长
* include_lengths：是否返回一个padded元组，因为有些时候会mask一些词，这时候需要知道哪些词被mask掉了
* batch_first：输入输出默认是\[seq_len, batch_size\]，设为True则是\[batch_size, seq_len\]，跟torch一样随性

**Dataset**

Dataset对象用来定义你的数据集，其实是继承了torch里的Dataset，这里会自动按Field里的方法对数据集进行处理。

常用的有TabularDataset，如果你是做机器翻译，可以用TranslationDataset。它们都可以通过splits同时生成训练集、验证集、测试集三个对象。

这里先举一个句子分类的例子。

In [19]:
from torchtext.data import Field, BucketIterator, TabularDataset

def tokenizer_jieba(text):
    return jieba.lcut(text)
    
TEXT = Field(tokenize=tokenizer_jieba, init_token='<sos>', eos_token='<eos>', lower=True)
LABEL = Field(sequential=False, use_vocab=False)

这里首先用定义了一个分词方法，输入输出分别是文本和分词后的列表。由于LABEL仅仅是标签，因此不需要额外的操作。

In [23]:
fields = [('text', TEXT), ('label', LABEL)]

train_data, val_data, test_data = TabularDataset.splits(path='data/', \
                                                             train='train.tsv', validation='dev.tsv', test='test.tsv', format='tsv', \
                                                             fields=fields)

print(vars(train_data.examples[0]))

fields就是前面定义好的Field，这里需要给每个Field取一个名字，用于内部的key-value储存，后面会用到。

TabularDataset的好处是它可以自动读取csv/tsv/json格式的数据集，其中每一行是一个pair。为了和前面BERT统一，这里也用tsv，但注意BERT会无视第一行，默认那里是header，torchtext不会。大家可以把之前BERT的数据集搬过来试一试，path是三者的公共路径，我这里是data，需要把三个文件放进去。

可以使用.examples来查看某一条sample，可以发现是一个字典，key就是fields里给的。

注意：这里的splits是cls，不会帮你做切分，你得事先分好数据集，切分的方法是split，按比例生成三份。

In [70]:
train_data, val_data, test_data = TabularDataset(path='data/data.tsv', format='tsv', fields=fields).split(split_ratio=[0.8, 0.1, 0.1])

这里我们还是继续用之前splits创建的对象。

In [25]:
TEXT.build_vocab(train_data, min_freq=10)

print(TEXT.vocab.stoi)
print(TEXT.vocab.itos)

随后就可以一句话生成vocab，min_freq是词最少出现的频率，低于它的词会被忽略掉。一般训练和测试都用的是训练集的vocab，不然测试的时候你会从和训练不一致的语料里生成，会很奇怪。

.vocab.stoi会返回word2id的字典，.itos则返回逆向映射id2word。

**Iterator**

Iterator对象是一个迭代容器，会自动帮你mini_batch，一般BuckeIterator用得比较多。

In [24]:
train_iter, val_iter, test_iter = BucketIterator.splits((train_data, val_data, test_data), batch_sizes=(128, 128, 128))

for i, batch in enumerate(train_iter):
    text = batch.text
    label = batch.label

注意：这里bacth的属性就是之前取的key。

接下来再看一个机器翻译的例子，TranslationDataset定义在datasets里。

In [27]:
from torchtext.datasets import TranslationDataset

SRC = Field(tokenize=tokenizer_jieba, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenizer_jieba, init_token='<sos>', eos_token='<eos>', lower=True)

exts = ('.src', '.trg')
fields = (SRC, TRG)

train_data, val_data, test_data = TranslationDataset.splits(path='data/', train='train/train', validation='val/val', \
                                                            test='test/test', exts=exts, fields=fields)

机器翻译里有source和target两句话，所以对应两个几乎相同的Field。exts是数据集后缀，我这里分别是`.src`和`.trg`，torchtext会去找对应的文件。fields这里不需要再取名，默认就是src和trg。

注意：这里的一堆路径刚上来可能看不懂，可以参照我data路径里的数据安放形式。

接下来就是差不多得生成vocab和batch。

In [24]:
SRC.build_vocab(train_data, min_freq=10)
TRG.build_vocab(train_data, min_freq=10)

train_iter, val_iter, test_iter = BucketIterator.splits((train_data, val_data, test_data), batch_size=128)

for i, batch in enumerate(train_iter):
    src = batch.src
    trg = batch.trg

总体来说不管是TabularDataset还是TranslationDataset，其实用起来都还挺简单的，主要是节约了很多时间~

以上就是基本操作了，接下来我们就可以调用torchtext的接口，把数据很方便地注入模型开始训练了。最后总结一下常用的接口，一般是这个顺序：
* Field：定义操作，分词、占位符等
* Dataset：定义数据，一般用splits创建对象，或者先创建对象再split切分
* build_vocab：创建词表
* Iterator：创建batch
* batch.xxx：搞定

torchtext+torch是我最常用的套路，这里非常推荐新手使用！

## 下期：教你入门机器翻译